In [4]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
# import seaborn as sns

from m import *


%matplotlib inline
sns.set_style("whitegrid")

#df= pd.read_csv('/data/DS_INTERN/data/RAW_DATA/train_test_data.csv', low_memory=False)
df = pd.read_csv('/data/DS_INTERN/data/RAW_DATA/train_test_data_new.csv', low_memory=False)

df1 = add_target_column(df)

train_df , test_df = train_test_split(df1)

train_df = perform_preprocessing(train_df)
test_df = perform_preprocessing(test_df)

In [3]:
len(train_df), len(test_df), len(df)

(50248, 244187, 294435)

In [ ]:
train_df = data_augmentation_simple_walmart(train_df, n_per_original=2)

# Trust Factor Calculation

In [18]:
X_train = train_df['FEAT1']
Y_train = train_df['TARGET']
X_test = test_df['FEAT1']

In [19]:
pipe= pipeline_voting_hard(max_iter=100)

In [20]:
pipe.fit(X_train, Y_train)

KeyboardInterrupt: 

In [42]:
from m import *

In [ ]:
Y_pred = pipe.predict(X_test)
#test_df['Predicted_target'] = pd.Series(Y_pred, index= test_df.index)

In [43]:
test_df.rename(columns={'TARGET':'Predicted_target'}, inplace= True)

In [44]:
item1_desc_mapping(train_df, test_df)

In [45]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244187 entries, 6537 to 294434
Data columns (total 3 columns):
ITEM1_DESC          244182 non-null object
FEAT1               244187 non-null object
Predicted_target    1770 non-null object
dtypes: object(3)
memory usage: 17.5+ MB


In [46]:
tgh = test_df.dropna()

In [47]:
tgi = trust_factor(tgh)

/libs/base/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/home/jupyter/m.py:643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_df['category'] = pd.Series(perform_spell_correction_manual(test_df['category']), index=test_df.index)
/home/jupyter/m.py:644: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [48]:
tgi.describe()

,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
count,1770.000000,1770.000000,1770.000000,1770.000000,1770.000000
mean,85.076271,97.587571,0.745198,0.961017,0.971751
std,25.755654,12.064808,0.435873,0.193609,0.165729
min,20.000000,21.000000,0.000000,0.000000,0.000000
25%,55.000000,100.000000,0.000000,1.000000,1.000000
50%,100.000000,100.000000,1.000000,1.000000,1.000000
75%,100.000000,100.000000,1.000000,1.000000,1.000000
max,100.000000,100.000000,1.000000,1.000000,1.000000


In [56]:
len(common_list), len(test_df)

(136, 248208)

In [29]:
# ITEM1_DESC mapping and updating the predicted_target

for i in range(len(common_list)):
    item = common_list[i]
    j = test_df.iloc[np.where(test_df['ITEM1_DESC']==item)].index
    for k in range(len(j)):
        test_df.loc[j[k]]['Predicted_target'] = train_df[train_df['ITEM1_DESC']==item]['TARGET'].iloc[0]

NameError: name 'common_list' is not defined

In [65]:
import Levenshtein as lev
from fuzzywuzzy import fuzz

test_df[['category','variety']] = test_df['Predicted_target'].str.split('_', expand=True)
#test_df.drop(['Predicted_target'], axis=  1, inplace = True)

test_df['category'] = pd.Series(perform_spell_correction_manual(test_df['category']), index=test_df.index)
test_df['variety'] = pd.Series(perform_spell_correction_manual(test_df['variety']), index=test_df.index)

test_df['category'] = pd.Series(perform_spell_correction_walmart(test_df['category']), index=test_df.index)
test_df['variety'] = pd.Series(perform_spell_correction_walmart(test_df['variety']), index=test_df.index)


test_df['category'] = pd.Series(perform_lemmatization(test_df['category']), index=test_df.index)
test_df['variety'] = pd.Series(perform_lemmatization(test_df['variety']), index=test_df.index)


test_df['fuzzy_category'] = test_df.apply(lambda x: fuzz.partial_token_set_ratio(x['FEAT1'],x['category']), axis=1)
test_df['fuzzy_variety'] = test_df.apply(lambda x: fuzz.partial_token_set_ratio(x['FEAT1'],x['variety']), axis=1)

test_df['TF_cat'] = test_df.apply(lambda x: 1 if x['fuzzy_category']>80 else 0, axis=1)
test_df['TF_var'] = test_df.apply(lambda x: 1 if x['fuzzy_variety']>80 else 0, axis=1)

test_df['TF'] = np.where((test_df['TF_cat'] == 1) | (test_df['TF_var'] ==1),1, 0)

In [66]:
test_df.describe()

,fuzzy_category,fuzzy_variety,TF_cat,TF_var,TF
count,248208.000000,248208.000000,248208.000000,248208.000000,248208.000000
mean,82.862764,89.733075,0.709893,0.809124,0.921320
std,27.506812,21.982778,0.453813,0.392992,0.269239
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50.000000,100.000000,0.000000,1.000000,1.000000
50%,100.000000,100.000000,1.000000,1.000000,1.000000
75%,100.000000,100.000000,1.000000,1.000000,1.000000
max,100.000000,100.000000,1.000000,1.000000,1.000000


In [5]:
tt = train_df.groupby(['ITEM1_DESC'], sort=False)['TARGET'].agg([('count_distinct_target', 'nunique')]).reset_index()

# select only the ones with count=1
ttt= tt[(tt['count_distinct_target']==1)]

# list of ITEM1_DESC
train_list_item = []
for i in range(len(ttt)):
    train_list_item.append(ttt.iloc[i]['ITEM1_DESC'])

#list of ITEM1_DESC for test
test_list_item = []
for i in range(len(test_df)):
    test_list_item.append(test_df.iloc[i]['ITEM1_DESC'])

common_list = list(set(train_list_item).intersection(set(test_list_item)))

len(common_list)


89

In [7]:
len(common_list)

89

In [12]:
len(train_df)

48197